In [19]:
import torch as torch
import torchvision as torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets.utils import download_and_extract_archive

import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
import os as re
import re as re

In [30]:
class MnistAugmentationDataset(Dataset):
    resources = [
        ("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "f68b3c2dcbeaaa9fbdd348bbdeb94873"),
        ("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz", "d53e105ee54ea40749a09fcbcd1e9432"),
        ("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", "9fb629c4189551a2d022fa330f9573f3"),
        ("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz", "ec29112dd5afa0611ce80d1b7f02629c")
    ]
    
    base_folder = '../data/MNIST/base'
    
    training_file = 'training.pt'
    test_file = 'test.pt'
    
    def download(self):
        for url, md5 in self.resources:
            filename = url.rpartition('/')[2]
            download_and_extract_archive(url, download_root=self.base_folder, filename=filename, md5=md5)
            
        for f in os.listdir(self.base_folder):
            if f[len(f) - 3: len(f)].lower() == '.gz':
                os.remove(os.path.join(self.base_folder, f))
    
    def __init__(self, name, seq, transform=None):
        self.root_dir = '../data/MNIST/augmented/' + name
        
        if not os.path.exists(self.root_dir):
            os.makedirs(self.root_dir)
        else:
            return
        
        if not os.path.exists(self.base_folder):
            os.makedirs(self.base_folder)
            self.download()
        
ds = MnistAugmentationDataset('test', None)